<a href="https://colab.research.google.com/github/calvinli2024/CS614-genai/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [1]:
%pip install evaluate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dataclasses import dataclass
from typing import Dict, List, Optional, Any

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    pipeline
)

import evaluate
from evaluate import evaluator

import itertools
import json
from copy import deepcopy

from peft import LoraConfig, TaskType, get_peft_model

2026-02-17 08:47:14.816413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771318035.160375      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771318035.250374      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771318036.009679      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771318036.009742      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771318036.009745      55 computation_placer.cc:177] computation placer alr

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Dataset

In [4]:
dataset = load_dataset("sh0416/ag_news")

dataset = {
    "train": dataset["train"].shuffle(seed=42).select(range(5000)),
    "valid": dataset["train"].shuffle(seed=42).select(range(5000, 6000)),
    "test": dataset["test"].shuffle(seed=42).select(range(2000))
}

num_labels = len(set(dataset["train"]["label"]))

README.md: 0.00B [00:00, ?B/s]

train.jsonl:   0%|          | 0.00/33.7M [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

# Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

# Ensure we have a padding token (Qwen is a causal LM)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

# Model

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    "Qwen/Qwen3-0.6B",
    num_labels=num_labels,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    pad_token_id=tokenizer.pad_token_id,
)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False, # set to False for training
    r=8, # dimension of the smaller matrices
    lora_alpha=32, # scaling factor
    lora_dropout=0.1 # dropout of LoRA layers
)

model = get_peft_model(model, lora_config)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Qwen3ForSequenceClassification(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024, padding_idx=151643)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
      

# Preprocess

In [7]:
max_length = 256

def preprocess(dataset_split, select_columns: List[str]):
    def run_tokenizer(row):
      return tokenizer(
          row["text"],
          padding="max_length",
          truncation=True,
          max_length=max_length,
      )

    def add_text(row):
      row["text"] = row["title"] + " " + row["description"]

      return row

    dataset_split = dataset_split.map(add_text)

    encoded_dataset = dataset_split.map(run_tokenizer, batched=True)

    encoded_dataset = encoded_dataset.remove_columns(
        [col for col in encoded_dataset.column_names if col not in select_columns]
    )

    return encoded_dataset.with_format("torch")

train_dataset = preprocess(dataset['train'], ["input_ids", "attention_mask", "label"])
valid_dataset = preprocess(dataset['valid'], ["input_ids", "attention_mask", "label"])
test_dataset = preprocess(dataset['test'], ["label", "text"])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Metrics

In [8]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    results = accuracy.compute(predictions=preds, references=labels)
    results.update(
        f1.compute(predictions=preds, references=labels, average="macro")
    )

    return results

# Hyperparameter Tuning

## Training Arguments

In [9]:
training_arguments = TrainingArguments(
    output_dir="./qwen3_agnews_grid",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=0,
    lr_scheduler_type="cosine",
    gradient_accumulation_steps=4,
    fp16=False, 
    report_to="none",
)

trainer_kwargs = dict(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

## Search Space

In [10]:
hyperparameters = {}

idx = 0
for learning_rate in [1e-4, 1e-3]:
    for num_train_epochs in [1, 2, 3]:
        hyperparameters[idx] = {
            "learning_rate": learning_rate,
            "num_train_epochs": num_train_epochs
        }

        idx += 1


## Helper Functions

In [11]:
def run_single_experiment(
    base_training_args: TrainingArguments,
    trainer_cls,
    trainer_kwargs: Dict[str, Any],
    hp_config: Dict[str, float|int],
    idx: int
) -> Dict[str, Any]:
    """
    base_training_args: a TrainingArguments object with default values.
    trainer_cls: usually `Trainer`.
    trainer_kwargs: dict with keys like model, train_dataset, eval_dataset, tokenizer, compute_metrics.
    hp_config: specific hyperparameters for this run (e.g. lr, epochs, batch size).
    """
    # 1) Clone TrainingArguments and override selected fields
    args_dict = base_training_args.to_dict()
    for k, v in hp_config.items():
        args_dict[k] = v

    training_args = TrainingArguments(**args_dict)

    # 2) Create a fresh Trainer (important if model should be reinitialized externally)
    trainer = trainer_cls(
        args=training_args,
        **trainer_kwargs,
    )

    # 3) Train and evaluate
    train_output = trainer.train()
    eval_metrics = trainer.evaluate()

    trainer.save_model(f"checkpoint_{idx}")

    result = {
        "hp_config": hp_config,
        "train_samples": train_output.metrics.get("train_samples", None),
        "eval_metrics": eval_metrics,
    }

    return result

def grid_search_hyperparams(
    base_training_args: TrainingArguments,
    trainer_cls,
    trainer_kwargs: Dict[str, Any],
    hyperparameters: Dict[int, Dict[str, int|float]],
    results_path: str = "grid_search_results.jsonl",
) -> List[Dict[str, Any]]:
    all_results = {}

    os.makedirs(os.path.dirname(results_path) or ".", exist_ok=True)

    with open(results_path, "w", encoding="utf-8") as f_out:
        for idx, combo in hyperparameters.items():
            print("\n=== Running config:", combo, "===")

            result = run_single_experiment(
                base_training_args=base_training_args,
                trainer_cls=trainer_cls,
                trainer_kwargs=deepcopy(trainer_kwargs),
                hp_config=combo,
                idx=idx
            )

            # Persist each result as one JSON line
            f_out.write(json.dumps(result) + "\n")
            f_out.flush()

            all_results[idx] = result

    return all_results

# Train

In [12]:
results = grid_search_hyperparams(
    base_training_args=training_arguments,
    trainer_cls=Trainer,
    trainer_kwargs=trainer_kwargs,
    hyperparameters=hyperparameters,
    results_path="grid_search_results.jsonl",
)


=== Running config: {'learning_rate': 0.0001, 'num_train_epochs': 1} ===


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


IndexError: Target 4 is out of bounds.

In [ ]:
best_config = max(results.items(), key=lambda r: r[1]["eval_metrics"].get("eval_f1", 0.0))

print("Best index:", best_config[0])
print("Best config:", best_config[1]["hp_config"])
print("Best metrics:", best_config[1]["eval_metrics"])

# Inference

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    f"./checkpoint_{best_config[0]}",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    pad_token_id=tokenizer.pad_token_id,
)

results = evaluator("text-classification").compute(
    model_or_pipeline=pipeline("text-classification", model=model),
    data=test_dataset,
    metric=evaluate.load("f1"),
    input_column="text",
    label_column="label",
    strategy="bootstrap",
    n_resamples=200
)

print(results)